Firstly, our Imports, the given functions and some global variables!

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
from moviepy.editor import VideoFileClip
from IPython.display import HTML
first_l=0
first_r=0
rho = 1 # distance resolution in pixels of the Hough grid
theta = np.pi/180 # angular resolution in radians of the Hough grid
threshold = 3     # minimum number of votes (intersections in Hough grid cell)
min_line_length = 5 #minimum number of pixels making up a line
max_line_gap = 1    # maximum gap in pixels between connectable line segments

filteredp3=[]    # list for the filter of the left lane
filteredp4=[]    # list for the filter of the right lane

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


# Draw Lines 
In Drawlines, firstly I divide the line points into two groups, that one set
has the points in the half left region defined by the vertices and the other in the other half right.
Moreover the points are checked if they are within a minimum distance on the x axis
from the previously tracked lane. The minimum distance starts from a smaller value for the points
near the top of the lane and becomes bigger in the bottom of the image. This comes from the fact 
that a bump will result in a bigger change on the x axis for the bottom than from the middle of the image
for the line tracked. Last but not least, the angle of the points is checked to be within the desirable limits. 

In addition, the tangent of their angles is checked if it is within a good limit
Left: lines_l_x, lines_l_y
Right: lines_r_y, lines_r_y

Then, I approximate the lanes by a 1st degree polynomial approximation on the two 
sets of lists.

Then I use a median filter for the angle and constants of the two lines:
Y=aX+b   a=median_filter(order 5)
         b=median_filter(order 5)


In [3]:

#Median filter for angles
def medfilt_a(arr):
    sum=0
    for each in arr:
        sum+=each[0]
    return sum/len(arr)

#Median filter for constants
def medfilt_b(arr):
    sum=0
    for each in arr:
        sum+=each[1]
    return sum/len(arr)

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    global first_l
    global first_r
    global filteredp3
    global filteredp4

    lines_l_x=[]
    lines_l_y=[]

    lines_r_x=[]
    lines_r_y=[]

    lastp4=[0,0]
    imshape=img.shape
    thres=imshape[1]/2
    th=10
    distance=25
    
    #Seperate Points in Left and Right group
    #constraints:
    #a) 0.7<(tangent(angle) < 2
    #b) distance from last lane approximation < distance+2*distance*(y-2*Ymax/3)/(Ymax/3) 
    for line in lines:
        for x1,y1,x2,y2 in line:
          if( abs((x2-x1)/(y2-y1))>0.7 and abs((x2-x1)/(y2-y1))<2 ):
           if(first_l==0):
             if(x1<thres and x2<thres):
               lines_l_x.append(x1)
               lines_l_x.append(x2)
               lines_l_y.append(y1)
               lines_l_y.append(y2) 
           else:
             if(x1<thres and x1<thres):
               lastp3=filteredp3[0]
               lastp3[0]=medfilt_a(filteredp3)
               lastp3[1]=medfilt_b(filteredp3)
               if(abs(int(lastp3(y1)-y1))<(distance+2*distance*(y1-2*imshape[0]/3)/(imshape[0]/3)) and
                  abs(int(lastp3(y2)-x2))<(distance+2*distance*(y1-2*imshape[0]/3)/(imshape[0]/3))):
                  lines_l_x.append(x1)
                  lines_l_x.append(x2)
                  lines_l_y.append(y1)
                  lines_l_y.append(y2)
           if(first_r==0):
             if(x1>thres and x2>thres):
               lines_r_x.append(x1)
               lines_r_x.append(x2)
               lines_r_y.append(y1)
               lines_r_y.append(y2)
           else:
             if(x1>thres and x2>thres):
               lastp4=filteredp4[0]
               lastp4[0]=medfilt_a(filteredp4)
               lastp4[1]=medfilt_b(filteredp4)
               if(abs(int(lastp4(y1)-x1))<(distance+2*distance*(y1-2*imshape[0]/3)/(imshape[0]/3)) and 
                  abs(int(lastp4(y2)-x2))<(distance+2*distance*(y1-2*imshape[0]/3)/(imshape[0]/3))):
                  lines_r_x.append(x1)
                  lines_r_x.append(x2)
                  lines_r_y.append(y1)
                  lines_r_y.append(y2)

    if(len(lines_l_x)>0):
       z3 = np.polyfit( lines_l_y,lines_l_x, 1)
       p3 = np.poly1d(z3)
       if(len(filteredp3)==0):
           filteredp3.append(p3)
           first_l=1
       else:
           filteredp3.append(p3)
           p3[0]=medfilt_a(filteredp3)
           p3[1]=medfilt_b(filteredp3)
       cv2.line(img,(int(p3(2*imshape[0]/3)),int(2*imshape[0]/3)), ( int(p3(imshape[0])) ,int(imshape[0]) ), color, thickness=th)
    elif( len(filteredp3)>=1):
       #Safety case : no points are extracted! use last approximation
       p3=filteredp3[0]
       p3[0]=medfilt_a(filteredp3)
       p3[1]=medfilt_b(filteredp3)
       cv2.line(img,(int(p3(2*imshape[0]/3)),int(2*imshape[0]/3)), ( int(p3(imshape[0])) ,int(imshape[0]) ), color, thickness=th)

    #Keep only 5 items for the filter
    if(len(filteredp3)==5): 
       del filteredp3[0]

    if(len(lines_r_x)>0):
       z4 = np.polyfit( lines_r_y,lines_r_x, 1)
       p4 = np.poly1d(z4)
       if( len(lines_r_x)==0):
           filteredp4.append(p4)
           first_r=1
       else:
           filteredp4.append(p4)
           p4[0]=medfilt_a(filteredp4)
           p4[1]=medfilt_b(filteredp4)
       cv2.line(img,(int(p4(2*imshape[0]/3)),int((2*imshape[0]/3))), (int(p4(imshape[0])),int(imshape[0])), color, thickness=th)
    elif( len(filteredp4)>=1):
       #Safety case : no points are extracted! use last approximation
       p4=filteredp4[0]
       p4[0]=medfilt_a(filteredp4)
       p4[1]=medfilt_b(filteredp4)
       cv2.line(img,(int(p4(2*imshape[0]/3)),int((2*imshape[0]/3))), (int(p4(imshape[0])),int(imshape[0])), color, thickness=th)
    
    #Keep only 5 items for the filter
    if(len(filteredp4)==5):
       del filteredp4[0]

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


# Findlane 
The main tricks here are:

a)The vertices of the region of interest is defined on proportion to the image axis sizes.

In [4]:

def findlane(image):
    imshape=image.shape
    gray=grayscale(image)
    gray_blur=gaussian_blur(image,7)
    edges = cv2.Canny(gray_blur, 100, 250)
    vertices = np.array([[(imshape[1]/8,int(imshape[0])),(imshape[1]/2-imshape[1]/6, 2*imshape[0]/3), 
                          (imshape[1]/2+imshape[1]/6, 2*imshape[0]/3), (7*imshape[1]/8+50,int(imshape[0]))]],
                        dtype=np.int32)
    roi=region_of_interest(edges,vertices)
    line_image = np.copy(image)*0 # creating a blank to draw lines on
    lines = hough_lines(roi, rho, theta, threshold, min_line_length, max_line_gap)
    color_edges = np.dstack((edges, edges, edges)) 
    lines_edges = cv2.addWeighted( image, 0.8,lines, 1, 0) 
    return lines_edges




def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)

    return findlane(image)


# Let's Test it!!!

In [5]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
first_l=0
first_r=0
filteredp3=[]
filteredp4=[] 
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████▉| 221/222 [00:12<00:00, 17.76it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 1min 15s, sys: 3.63 s, total: 1min 18s
Wall time: 13.2 s


In [6]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output)) 

In [7]:
first_l=0
first_r=0
filteredp3=[]
filteredp4=[] 
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|█████████▉| 681/682 [00:40<00:00, 17.62it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 3min 53s, sys: 11.4 s, total: 4min 4s
Wall time: 40.8 s


In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [9]:
first_l=0
first_r=0
filteredp3=[]
filteredp4=[] 
extra= 'extra.mp4'
clip3 = VideoFileClip('challenge.mp4')
yellow_clip = clip3.fl_image(process_image)
%time yellow_clip.write_videofile(extra, audio=False)

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|██████████| 251/251 [00:26<00:00,  9.65it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

CPU times: user 2min 18s, sys: 6.8 s, total: 2min 25s
Wall time: 27.3 s


In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(extra))

# Improvements 


1)Better Filter:

   A butterworth filter might be better in following immediate true changes like when bumps occur.
   
2)Bump detection function to help Filter weight the latest change value more than the past. 

3)More Test videos are required to experiment on the parameters like tangent(angle) limits, blurring,
  can

 